In [6]:
import pandas as pd 
import os
from sqlalchemy import create_engine

In [7]:
engine= create_engine('sqlite:///inventory.db')

In [ ]:
import pandas as pd 
import os
from sqlalchemy import create_engine

# 1. Define the Database Engine (assuming this is done earlier in your script)
# Replace the URI with your actual database connection string
engine = create_engine('sqlite:///inventory.db') 

# 2. Set the chunk size for both reading and writing
# 50,000 rows is a good starting point for large files
CHUNK_SIZE = 50000 

for file in os.listdir('data'):
    if '.csv' in file:
        file_path = os.path.join('data', file)
        table_name = file[:-4]  # Remove the last 4 characters (.csv)
        
        # This flag controls whether to replace the table or append to it
        if_exists_mode = 'replace' 
        
        print(f"\n--- Starting Ingestion: {file} into table '{table_name}' ---")
        
        try:
            # A. Read the CSV in chunks (pd.read_csv returns an iterator)
            # This is the main change to fix the MemoryError during file reading.
            chunk_iterator = pd.read_csv(file_path, chunksize=CHUNK_SIZE)
            
            for i, chunk_df in enumerate(chunk_iterator):
                print(f"  -> Processing chunk {i+1} with {len(chunk_df)} rows...")
                
                # B. Ingest the current chunk into the database
                chunk_df.to_sql(
                    name=table_name,
                    con=engine,
                    if_exists=if_exists_mode, # Use 'replace' for the first chunk, then 'append'
                    index=False
                )
                
                # C. Switch the mode to 'append' for all subsequent chunks
                if_exists_mode = 'append'
                
            print(f"--- Successfully finished {file} ---")

        except Exception as e:
            # Catching errors related to the file or connection
            print(f"!!! Error processing file {file}: {e}")

: 